In [1]:
import sys
sys.path.append('../')
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
from config import chromeSetup,get_selected_cols_df,ensure_dir,get_Drivers,encode_search_word,enter_sys_value,update_iod_tms_with_now,transform
import warnings
import shutil
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotVisibleException, ElementNotSelectableException
import selenium.webdriver.support.expected_conditions as EC
import time 
from datetime import datetime
from glob import glob
from tqdm.notebook import tqdm as tqdm_notebook
import json
import os
import requests
from collections import ChainMap

In [2]:
from config import Marco

marco = Marco()
marco.colsSend2Carrier

['Sales',
 'Load ID',
 'Stop Seq',
 'Carrier Code',
 'Carrier Name',
 'Service Type',
 'Alert Type',
 'Ship Date',
 'ETA',
 'From Address',
 'To Address',
 'Customer Name',
 'GERP Order No',
 'Pick Order No',
 'Pro No']

In [3]:
timestamp = str(datetime.now())        

today_date = timestamp[:10]
today_date




'2021-06-07'

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
browers = webdriver.Chrome("C:\\Users\\marco.wang\\Downloads\\chromedriver_win32\\chromedriver")

In [5]:
 print("get browsers!!")
log_in_url = 'https://gtli.mercurygate.net/MercuryGate/login/spLogin.jsp?Timeout=True&returnUrl=..%2Fmainframe%2FMainFrame.jsp&bCompaySelected=false'
browers.get(log_in_url)
time.sleep(3)



### click reference and select tracjking date 

get browsers!!


In [6]:
browers.switch_to.default_content()
browers.switch_to.frame("AppBody")
browers.switch_to.frame(browers.find_element_by_id('Detail'))
browers.switch_to.frame("winportletListShipmentCustomerManageShipmentsIFrame")


FEDEX_CARRIER_CODE = ['GLBL','GLPC']
FILE_PATH = sorted(glob("..\\daily_route_data\\IOD_MONITORING_DETAIL_US_*.csv"))[-1]
print(FILE_PATH)
df_origin = pd.read_csv(FILE_PATH)
dit = {}
df = df_origin[df_origin['Carrier Code'].isin(FEDEX_CARRIER_CODE)]
for load in tqdm_notebook(df['Load ID']):
    IOD = browers.find_element_by_xpath("//*[@id='sReference']")
    IOD.clear()
    IOD.send_keys(load)
    time.sleep(0.5)
    find = browers.find_element_by_xpath("//*[@id='shipmentListForm']/table[1]/tbody/tr[2]/td[5]/input")
    find.click()
    time.sleep(3)
    Status = browers.find_element_by_xpath("/html/body/form/table[2]/tbody/tr[2]/td[2]")
    Origin = browers.find_element_by_xpath("/html/body/form/table[2]/tbody/tr[2]/td[3]")
    Destination = browers.find_element_by_xpath("/html/body/form/table[2]/tbody/tr[2]/td[4]")
    Drop_Appt = browers.find_element_by_xpath("/html/body/form/table[2]/tbody/tr[2]/td[5]")
    dit[load] = {
        "Status":Status.text,
        "Origin":Origin.text,
        "Destination":Destination.text,
        "Drop_Appt":"No APPT" if Drop_Appt.text == ' ' else Drop_Appt.text
    }
    
    

..\daily_route_data\IOD_MONITORING_DETAIL_US_2021-06-07 17-22.csv


WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.77)


In [9]:
timestamp = "".join([d for d  in timestamp if d.isdigit()])
ensure_dir(os.path.join("table_result",today_date))
df['Status_'] = df['Load ID'].apply(lambda x : dit[x]['Status'] if dit.get(x) else "NA")
df['Stop Seq'] = df['Load ID'].apply(lambda x : dit[x]['Drop_Appt'] if dit.get(x) else "NA")
df = df[marco.colsSend2Carrier]
df.rename(columns= {
    'Stop Seq':"DROP APPT"
},inplace = True)
df = df.sort_values("DROP APPT")

C:\Users\marco.wang\Miniconda3\envs\pantos\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\marco.wang\Miniconda3\envs\pantos\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
df.to_excel(ensure_dir(os.path.join("table_result",today_date,f"report_GLBL_{timestamp[8:12]}.xlsx"),index=False,engine='xlsxwriter')